In [1]:
%pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
%pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [3]:
%pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Data processing
import pandas as pd
import numpy as np


import tensorflow as tf

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os,random,math
TRAINING_DIR="/content/drive/MyDrive/propaganda_dataset_v2"
files=os.listdir(TRAINING_DIR)

traindf= pd.read_csv(os.path.join(TRAINING_DIR,files[0]),sep = '\t')
testdf= pd.read_csv(os.path.join(TRAINING_DIR,files[1]),sep = '\t')

In [8]:
def replace_token(sentences):
    start_sent=sentences.split("<BOS>")
    end_sent=start_sent[1].split("<EOS>")
    return end_sent[0]

testdf["tagged_in_context"]=testdf["tagged_in_context"].map(replace_token)
traindf["tagged_in_context"]=traindf["tagged_in_context"].map(replace_token)

testdf.head(10)

,label,tagged_in_context
0,not_propaganda,according to a UN estimate.
1,causal_oversimplification,the country would not last long without an ou...
2,appeal_to_fear_prejudice,gets Earl Warren and Sen. Richard Russel to j...
3,not_propaganda,You
4,repetition,infidels
5,"name_calling,labeling","the ""gay lifestyle"
6,loaded_language,devastating communities
7,not_propaganda,Jacob
8,flag_waving,Iran’s long rap sheet of aggression against A...
9,doubt,"Now, the pope’s reply to my testimony was: “I..."


In [9]:
traindf["label"].value_counts(dropna=False)

not_propaganda               1191
exaggeration,minimisation     164
causal_oversimplification     158
name_calling,labeling         157
loaded_language               154
appeal_to_fear_prejudice      151
flag_waving                   148
repetition                    147
doubt                         144
Name: label, dtype: int64

In [10]:
def replace_tokens_multiclass(label):
    if label=="flag_waving":
        return 0
    elif label=="appeal_to_fear_prejudice":
        return 1
    elif label=="causal_oversimplification":
        return 2
    elif label =="doubt":
        return 3
    elif label == "exaggeration,minimisation":
        return 4
    elif label == "loaded_language":
        return 5
    elif label == "name_calling,labeling":
        return 6
    elif label == "repetition":
        return 7
     
#drop rows with no propanganda

testdf=testdf[testdf.label != "not_propaganda"]
traindf = traindf[traindf.label != "not_propaganda"]

traindf["label"] = traindf["label"].map(replace_tokens_multiclass)
testdf["label"] = testdf["label"].map(replace_tokens_multiclass)

traindf.reset_index(inplace=True)
testdf.reset_index(inplace = True)


<ipython-input-10-d738dd8bbd4c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindf["label"] = traindf["label"].map(replace_tokens_multiclass)
<ipython-input-10-d738dd8bbd4c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["label"] = testdf["label"].map(replace_tokens_multiclass)


In [11]:
traindf

,index,label,tagged_in_context
0,2,0,American people
1,5,5,annihilated
2,8,3,so-called evidence
3,10,6,hateful conduct
4,12,1,point to Iran’s positioning itself for more a...
...,...,...,...
1218,2403,7,Nazi
1219,2405,4,absolutely no place for anti-Semitism
1220,2406,0,Prosecutors Doing Mueller’s ‘Dirty Work Are A...
1221,2407,2,Neither the Democrat leadership nor the Democ...


In [12]:
traindf["label"].value_counts(dropna=False)

4    164
2    158
6    157
5    154
1    151
0    148
7    147
3    144
Name: label, dtype: int64

In [13]:
hg_train_data = Dataset.from_pandas(traindf)
hg_test_data = Dataset.from_pandas(testdf)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_data(data):
    return tokenizer(data["tagged_in_context"],
                     max_length=140,
                     truncation=True,
                     padding="max_length")



In [15]:
dataset_train = hg_train_data.map(tokenize_data)
dataset_test = hg_test_data.map(tokenize_data)

Map:   0%|          | 0/1223 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=8)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [17]:
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=7,              
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)



In [18]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.940500,1.661399,0.394265
2,1.352800,1.375340,0.541219
3,0.916100,1.323023,0.573477
4,0.659500,1.396038,0.580645


TrainOutput(global_step=4892, training_loss=1.2172387934861764, metrics={'train_runtime': 476.5401, 'train_samples_per_second': 17.965, 'train_steps_per_second': 17.965, 'total_flos': 351971107864320.0, 'train_loss': 1.2172387934861764, 'epoch': 4.0})

In [20]:
# Predictions
y_test_predict = trainer.predict(dataset_test)
y_test_logits = y_test_predict.predictions


In [21]:
y_test_probabilities = tf.nn.softmax(y_test_logits)
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 8), dtype=float32, numpy=
array([[4.44889039e-01, 3.53510022e-01, 3.73906791e-02, 4.52662446e-02,
        7.07438141e-02, 1.19348308e-02, 9.97083634e-03, 2.62945406e-02],
       [1.88786052e-02, 1.91252440e-01, 6.40626550e-01, 7.32262880e-02,
        1.50916502e-02, 7.06236251e-03, 1.34446155e-02, 4.04175073e-02],
       [1.11520116e-03, 1.45235786e-03, 9.28990077e-04, 1.40813261e-03,
        3.01252375e-03, 2.65316456e-03, 9.86726880e-01, 2.70270975e-03],
       [3.43217589e-02, 9.67723131e-03, 9.34087764e-03, 4.83229337e-03,
        9.40849409e-02, 1.19504929e-01, 5.83737254e-01, 1.44500732e-01],
       [6.28511310e-02, 4.43309993e-02, 1.18490933e-02, 7.32110860e-03,
        5.49580939e-02, 7.22514465e-02, 1.13052009e-02, 7.35132933e-01]],
      dtype=float32)>

In [22]:
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

In [23]:
y_test_actual_labels = y_test_predict.label_ids

In [24]:
y_test_actual_labels[:5]

array([2, 1, 7, 6, 5])

In [25]:
trainer.evaluate(dataset_test)

{'eval_loss': 1.323022723197937,
 'eval_accuracy': 0.5734767025089605,
 'eval_runtime': 4.58,
 'eval_samples_per_second': 60.918,
 'eval_steps_per_second': 60.918,
 'epoch': 4.0}

In [33]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

f1 = f1_score(y_test_actual_labels, y_test_pred_labels, average='weighted', zero_division=1)
precision = precision_score(y_test_actual_labels, y_test_pred_labels, average='weighted', zero_division=1)
accuracy = accuracy_score(y_test_actual_labels, y_test_pred_labels)
recall = recall_score(y_test_actual_labels, y_test_pred_labels, average='weighted', zero_division=1)
# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.5734767025089605
Precision: 0.5674731623233454
Recall: 0.5734767025089605
F1 score: 0.564617953630277


In [27]:

from sklearn.metrics import classification_report

target_names=["0","1","2","3","4","5","6","7"]
# Compute classification report
report = classification_report(y_test_actual_labels, y_test_pred_labels, target_names= target_names)

# Print classification report
print(report)

              precision    recall  f1-score   support

           0       0.64      0.69      0.67        39
           1       0.65      0.65      0.65        43
           2       0.60      0.48      0.54        31
           3       0.55      0.74      0.63        38
           4       0.55      0.43      0.48        28
           5       0.51      0.59      0.55        37
           6       0.61      0.65      0.62        31
           7       0.40      0.25      0.31        32

    accuracy                           0.57       279
   macro avg       0.56      0.56      0.56       279
weighted avg       0.57      0.57      0.56       279



In [28]:
# Save tokenizer_to_our_system
tokenizer.save_pretrained('./sentiment_transfer_learning_transformer/')

# Save model to out system
trainer.save_model('./sentiment_transfer_learning_transformer/')